In [0]:
# COMMAND ----------#
# 1. ライブラリの準備
# Databricks Runtime 14.x 以降推奨
%pip install --upgrade databricks-langchain databricks-vectorsearch mlflow langgraph pillow sentence-transformers
dbutils.library.restartPython()

In [0]:
# COMMAND ----------
import os
import tempfile
import mlflow
import json
import numpy as np
import warnings
import uuid
from typing import List, Any, Generator, Optional, Sequence, Union

# 必要なライブラリのインポート
from databricks_langchain import (
    ChatDatabricks,
    VectorSearchRetrieverTool,
    DatabricksFunctionClient,
    UCFunctionToolkit,
    set_uc_function_client,
)

from langchain_core.embeddings import Embeddings
from langchain_core.language_models import LanguageModelLike
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langgraph.graph import END, StateGraph
from langgraph.prebuilt.tool_node import ToolNode
from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import (
    ChatAgentChunk,
    ChatAgentMessage,
    ChatAgentResponse,
    ChatContext,
)

# sentence-transformers のインポート（警告を抑制）
warnings.filterwarnings("ignore", message=".*use_fast.*")
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from sentence_transformers import SentenceTransformer

from PIL import Image, ImageOps

print("✅ すべてのライブラリが正常にインポートされました")


In [0]:
# COMMAND ----------
# 定数定義

# Widgetsの作成
dbutils.widgets.text("catalog", "fall_detection_demo_catalog", "カタログ")
dbutils.widgets.text("schema", "{ご自身のスキーマ名を入力}", "スキーマ")
dbutils.widgets.text("suffix", "{ご自身のSuffixを指定}", "ENDPOINT用の接尾辞")
dbutils.widgets.dropdown("recreate_schema", "False", ["True", "False"], "スキーマを再作成")


# Widgetからの値の取得
CATALOG = dbutils.widgets.get("catalog")
SCHEMA = dbutils.widgets.get("schema")
RECREATE_SCHEMA = dbutils.widgets.get("recreate_schema") == "True"
SUFFIX = dbutils.widgets.get("suffix")

# Volume パス定義
VIDEO_VOL = f"/Volumes/{CATALOG}/{SCHEMA}/video_volume"
FRAME_VOL = f"/Volumes/{CATALOG}/{SCHEMA}/frame_volume"

INDEX_NAME = f"{CATALOG}.{SCHEMA}.fall_detection_index"

VS_ENDPOINT = f"fall_detection_vector_search_{SUFFIX}"
LLM_ENDPOINT = "databricks-claude-3-7-sonnet"


# MLflow 自動ログ有効化
mlflow.langchain.autolog()
client = DatabricksFunctionClient()
set_uc_function_client(client)

# CLIP モデルの初期化
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    clip_model = SentenceTransformer('clip-ViT-B-32')

print("✅ CLIP モデルが正常に初期化されました")


In [0]:
# COMMAND ----------#
mlflow.langchain.autolog()
client = DatabricksFunctionClient()
set_uc_function_client(client)


In [0]:
# COMMAND ----------
# 修正版: Embeddings 基底クラスを継承したCLIPエンベディング
class CLIPTextEmbeddings(Embeddings):
    def __init__(self, model):
        super().__init__()
        self.model = model
    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            return [self.model.encode(text).tolist() for text in texts]
    
    def embed_query(self, text: str) -> List[float]:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            return self.model.encode(text).tolist()

clip_embeddings = CLIPTextEmbeddings(clip_model)

# VectorSearchRetrieverTool の作成
vs_tool = VectorSearchRetrieverTool(
    index_name        = INDEX_NAME,
    endpoint_name     = VS_ENDPOINT,
    embedding         = clip_embeddings,
    text_column       = "image_path",  # 必須パラメータ
    columns           = ["image_path"],
    num_results       = 5,
    tool_name         = f"fall_detection_image_search_{SUFFIX}",
    tool_description  = (
        "防犯カメラのフレーム画像を自然言語クエリで検索し、"
        "類似度の高い画像パスを返します。"
    ),
)

print("✅ VectorSearchRetrieverTool が正常に作成されました")


In [0]:
# COMMAND ----------
# 修正版: text_column パラメータを追加
try:
    vs_tool = VectorSearchRetrieverTool(
        index_name        = INDEX_NAME,
        endpoint_name     = VS_ENDPOINT,
        embedding         = clip_embeddings,
        text_column       = "image_path",          # ← 追加: 検索対象のテキストカラム
        columns           = ["image_path"],
        num_results       = 5,
        tool_name         = "fall_detection_image_search",
        tool_description  = (
            "防犯カメラのフレーム画像を自然言語クエリで検索し、"
            "類似度の高い画像パスを返します。"
        ),
    )
    print("✅ VectorSearchRetrieverTool が正常に作成されました")
    
except Exception as e:
    print(f"❌ エラーが発生しました: {str(e)}")
    import traceback
    traceback.print_exc()


In [0]:
# COMMAND ----------
# 完全修正版: OpenAI形式メッセージ変換対応エージェント
class LangGraphChatAgent(ChatAgent):
    """OpenAI形式メッセージ変換に対応した LangGraph ChatAgent"""
    
    def __init__(self, agent_graph):
        self.agent_graph = agent_graph

    def _generate_unique_id(self) -> str:
        """一意な ID を生成"""
        return str(uuid.uuid4())

    def _convert_to_openai_format(self, messages: List[ChatAgentMessage]) -> List[dict]:
        """ChatAgentMessage を OpenAI 形式の辞書に変換"""
        openai_messages = []
        
        for msg in messages:
            # OpenAI 形式: {'role': 'user/assistant/system', 'content': '...'}
            openai_msg = {
                "role": msg.role,
                "content": msg.content or "",
            }
            
            # オプションフィールドの追加
            if msg.name:
                openai_msg["name"] = msg.name
            if msg.tool_calls:
                openai_msg["tool_calls"] = msg.tool_calls
            if msg.tool_call_id:
                openai_msg["tool_call_id"] = msg.tool_call_id
                
            openai_messages.append(openai_msg)
        
        return openai_messages

    def _convert_from_langchain_message(self, lc_msg_dict: dict) -> ChatAgentMessage:
        """LangChain メッセージ辞書を ChatAgentMessage に変換"""
        
        # LangChain の message_to_dict 形式: {'type': 'human', 'data': {...}}
        if 'type' in lc_msg_dict and 'data' in lc_msg_dict:
            msg_type = lc_msg_dict['type']
            msg_data = lc_msg_dict['data']
            
            # type を role にマッピング
            role_mapping = {
                'human': 'user',
                'ai': 'assistant', 
                'system': 'system',
                'tool': 'tool'
            }
            
            role = role_mapping.get(msg_type, 'assistant')
            content = msg_data.get('content', '')
            
            return ChatAgentMessage(
                id=msg_data.get('id') or self._generate_unique_id(),
                role=role,
                content=content,
                name=msg_data.get('name'),
                tool_calls=msg_data.get('tool_calls'),
                tool_call_id=msg_data.get('tool_call_id'),
                attachments=msg_data.get('attachments')
            )
        
        # 直接的な辞書形式の場合
        else:
            return ChatAgentMessage(
                id=lc_msg_dict.get('id') or self._generate_unique_id(),
                role=lc_msg_dict.get('role', 'assistant'),
                content=lc_msg_dict.get('content', ''),
                name=lc_msg_dict.get('name'),
                tool_calls=lc_msg_dict.get('tool_calls'),
                tool_call_id=lc_msg_dict.get('tool_call_id'),
                attachments=lc_msg_dict.get('attachments')
            )

    def predict(self, messages: List[ChatAgentMessage], context: ChatContext = None, custom_inputs=None) -> ChatAgentResponse:
        """修正版 predict メソッド: OpenAI形式変換対応"""
        try:
            # ChatAgentMessage を OpenAI 形式に変換
            openai_messages = self._convert_to_openai_format(messages)
            request = {"messages": openai_messages}
            
            print(f"🔄 OpenAI形式メッセージ: {json.dumps(openai_messages, ensure_ascii=False, indent=2)}")
            
            all_messages = []
            for event in self.agent_graph.stream(request, stream_mode="updates"):
                print(f"📨 Event: {event}")
                
                for node_name, node_data in event.items():
                    if node_data and "messages" in node_data:
                        for msg_data in node_data["messages"]:
                            if msg_data is not None:
                                try:
                                    # LangChain メッセージを ChatAgentMessage に変換
                                    chat_msg = self._convert_from_langchain_message(msg_data)
                                    all_messages.append(chat_msg)
                                    print(f"✅ 変換成功: {chat_msg.role} - {chat_msg.content[:100]}...")
                                except Exception as conv_error:
                                    print(f"⚠️ メッセージ変換エラー: {conv_error}")
                                    # フォールバック: 基本的なアシスタントメッセージを作成
                                    fallback_msg = ChatAgentMessage(
                                        id=self._generate_unique_id(),
                                        role="assistant",
                                        content=str(msg_data)
                                    )
                                    all_messages.append(fallback_msg)
            
            if not all_messages:
                # メッセージが一つもない場合のフォールバック
                fallback_msg = ChatAgentMessage(
                    id=self._generate_unique_id(),
                    role="assistant",
                    content="画像検索が完了しましたが、結果の取得に問題が発生しました。"
                )
                all_messages.append(fallback_msg)
            
            return ChatAgentResponse(messages=all_messages)
            
        except Exception as e:
            print(f"❌ エージェント実行エラー: {str(e)}")
            import traceback
            traceback.print_exc()
            
            # エラー時のフォールバック応答（ID 付き）
            error_message = ChatAgentMessage(
                id=self._generate_unique_id(),
                role="assistant",
                content=f"画像検索中にエラーが発生しました: {str(e)}"
            )
            return ChatAgentResponse(messages=[error_message])

print("✅ LangGraphChatAgent クラスが作成されました")


In [0]:
# COMMAND ----------
# エージェントの構築
tools = [vs_tool]
llm = ChatDatabricks(endpoint=LLM_ENDPOINT)
system_prompt = (
    "あなたは防犯カメラ画像検索のアシスタントです。"
    "ユーザーが入力した自然言語を使って画像を検索し、"
    "最も関連する画像パスを返してください。"
)

def create_tool_calling_agent(
    model: LanguageModelLike,
    tools: Sequence[ToolNode],
    system_prompt: Optional[str] = None,
):
    bound_model = model.bind_tools(tools)

    def should_continue(state: ChatAgentState):
        last = state["messages"][-1]
        return "continue" if last.get("tool_calls") else "end"

    preproc = (
        RunnableLambda(lambda s: [{"role": "system", "content": system_prompt}] + s["messages"])
        if system_prompt else
        RunnableLambda(lambda s: s["messages"])
    )
    model_runnable = preproc | bound_model

    def call_model(state: ChatAgentState, config: RunnableConfig):
        resp = model_runnable.invoke(state, config)
        return {"messages": [resp]}

    sg = StateGraph(ChatAgentState)
    sg.add_node("agent", RunnableLambda(call_model))
    sg.add_node("tools", ChatAgentToolNode(tools))
    sg.set_entry_point("agent")
    sg.add_conditional_edges("agent", should_continue, {"continue": "tools", "end": END})
    sg.add_edge("tools", "agent")
    return sg.compile()

# エージェントの作成
agent_graph = create_tool_calling_agent(llm, tools, system_prompt)
AGENT = LangGraphChatAgent(agent_graph)

# MLflow モデルとしてログ
mlflow.models.set_model(AGENT)

print("✅ エージェントが正常に作成されました")


In [0]:
# COMMAND ----------
# 修正版テストコード（画像表示機能付き）
import json
import re
from PIL import Image
import os

query_text = "person lying on the floor"

with mlflow.start_run(run_name="image_search_with_display") as run:
    try:
        # ユーザーメッセージの作成（ID 付き）
        user_msg = ChatAgentMessage(
            id=str(uuid.uuid4()),
            role="user",
            content=query_text
        )
        print(f"✅ ユーザーメッセージ作成: ID={user_msg.id}")
        
        # エージェント呼び出し
        print("🔄 エージェント呼び出し開始...")
        response = AGENT.predict(messages=[user_msg])
        
        if response and response.messages:
            print(f"✅ 応答メッセージ数: {len(response.messages)}")
            
            # 検索結果から画像パスを抽出
            image_paths = []
            
            for i, msg in enumerate(response.messages):
                print(f"📋 メッセージ{i+1}: ID={msg.id}, Role={msg.role}")
                print(f"   内容: {msg.content[:200]}...")
                
                # メッセージ内容から画像パスを抽出
                if msg.role == "assistant" and msg.content:
                    # JSONフォーマットの検索結果を解析
                    try:
                        # JSONとして解析を試行
                        if msg.content.strip().startswith('[') or msg.content.strip().startswith('{'):
                            parsed_content = json.loads(msg.content)
                            if isinstance(parsed_content, list):
                                for item in parsed_content:
                                    if isinstance(item, dict) and 'image_path' in item:
                                        image_paths.append(item['image_path'])
                            elif isinstance(parsed_content, dict) and 'image_path' in parsed_content:
                                image_paths.append(parsed_content['image_path'])
                    except json.JSONDecodeError:
                        # JSON解析に失敗した場合、正規表現でパスを抽出
                        pass
                    
                    # 正規表現でファイルパスを抽出（/で始まり.jpgで終わるパターン）
                    path_pattern = r'/[^\s]*\.(?:jpg|jpeg|png|gif|bmp)'
                    found_paths = re.findall(path_pattern, msg.content, re.IGNORECASE)
                    image_paths.extend(found_paths)
                    
                    # フレームボリューム内のパスパターンも検索
                    frame_vol_pattern = r'/Volumes/[^\s]*\.(?:jpg|jpeg|png|gif|bmp)'
                    frame_paths = re.findall(frame_vol_pattern, msg.content, re.IGNORECASE)
                    image_paths.extend(frame_paths)
            
            # 重複を除去
            image_paths = list(set(image_paths))
            
            assistant = response.messages[-1]
            
            # MLflow ログ
            mlflow.log_param("query_text", query_text)
            mlflow.log_param("response_status", "success")
            mlflow.log_param("agent_response", assistant.content)
            mlflow.log_param("message_count", len(response.messages))
            mlflow.log_param("embedding_model", "clip-ViT-B-32")
            mlflow.log_param("found_image_count", len(image_paths))
            mlflow.log_param("image_paths", ";".join(image_paths))
            
            # 検索結果の画像をすべて表示
            print(f"\n🖼️ 検索された画像数: {len(image_paths)}")
            
            if image_paths:
                for idx, img_path in enumerate(image_paths, 1):
                    try:
                        if os.path.exists(img_path):
                            print(f"\n📷 画像 {idx}: {img_path}")
                            
                            # 画像をロードして表示
                            img = Image.open(img_path)
                            
                            # 画像のサイズ情報を表示
                            print(f"   サイズ: {img.size[0]} x {img.size[1]} ピクセル")
                            
                            # Databricks ノートブックに画像を表示
                            display(img)
                            
                            # サムネイル作成（MLflow用）
                            thumbnail = img.copy()
                            thumbnail.thumbnail((300, 300))  # 300x300のサムネイル
                            
                            # 一時ディレクトリにサムネイルを保存
                            thumb_filename = f"thumbnail_{idx}_{os.path.basename(img_path)}"
                            thumb_path = os.path.join("/tmp", thumb_filename)
                            thumbnail.save(thumb_path)
                            
                            # MLflowにサムネイルをログ
                            mlflow.log_artifact(thumb_path, artifact_path="search_result_thumbnails")
                            
                        else:
                            print(f"⚠️ 画像ファイルが見つかりません: {img_path}")
                            
                    except Exception as img_error:
                        print(f"❌ 画像表示エラー ({img_path}): {str(img_error)}")
                
                # 画像一覧をMarkdown形式でも表示
                print("\n## 検索結果画像一覧")
                for idx, img_path in enumerate(image_paths, 1):
                    print(f"{idx}. `{img_path}`")
                    
            else:
                print("❌ 検索結果に画像パスが見つかりませんでした")
                print("デバッグ用: エージェントの応答内容:")
                for msg in response.messages:
                    print(f"  - {msg.role}: {msg.content}")
            
            print(f"\n✅ MLflow run: {mlflow.get_artifact_uri()}")
            
        else:
            print("❌ エージェントからの応答がありません")
            mlflow.log_param("response_status", "no_response")
            
    except Exception as e:
        print(f"❌ エラー発生: {str(e)}")
        mlflow.log_param("error_message", str(e))
        mlflow.log_param("response_status", "error")
        import traceback
        traceback.print_exc()
